In [16]:
# 重新加载salary计算模块
import importlib
importlib.reload(commodity)

<module 'commodity' from '/Volumes/Repository/Projects/ffa/commodity.py'>

In [2]:
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import pandas as pd
import dash_mantine_components as dmc
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import commodity
import akshare as ak
from datetime import datetime, timedelta

app = Dash(external_stylesheets=[dbc.themes.FLATLY])

# 创建主品种数据
symbol_id = 'RB'
symbol_name = '螺纹钢'
fBasePath = 'steel/data/mid-stream/螺纹钢/'
json_file = './steel/setting.json'
chain_id = 'steel'
chain_name = '黑色金属'
symbol = commodity.SymbolData(symbol_id, symbol_name, json_file)
symbol_chain = commodity.SymbolChain(chain_id, chain_name, json_file)
page_property = {
    'symbol_id': 'RB',
    'symbol_name': '螺纹钢',
    'json_file': './steel/setting.json',
    'json_file': './steel/setting.json',
    'chain_id': 'steel',
    'chain_name': '黑色金属',
    'symbol': {},
    'symbol_chain': {},
    'symbol_figure': {},
    'chart_setting': {},
}

# 初始化数据
def initial_data():
    # 构造品种数据访问对象
    # symbol = commodity.SymbolData(symbol_id, symbol_name, json_file)
    symbol_j = commodity.SymbolData('J', '焦炭', json_file)
    symbol_i = commodity.SymbolData('I', '铁矿石', json_file)
    symbol_chain.add_symbol(symbol)
    symbol_chain.add_symbol(symbol_j)
    symbol_chain.add_symbol(symbol_i)
    symbol_chain.initialize_data()
    symbol.get_spot_months()
    df_profit = symbol.get_profits(symbol_chain)    

# # 主图全局变量
# main_figure = {}
charts_setting_dict = {}

# 基本面分析配置面板
main_chart_config =dbc.Accordion(
    [
        dbc.AccordionItem(
            [                
                dbc.Label('选择分析指标：', color='darkblue'),
                dbc.Checklist(
                    options=['基差率', '库存', '仓单', '库存消费比', '库存+仓单', '现货利润', '盘面利润', '现货利润+盘面利润'],
                    value=['基差率', '库存', '仓单', '现货利润', '盘面利润'],
                    id='select_index', inline=True
                ),
                html.Hr(),
                dbc.Label('标记区间：', color='darkblue'),
                dbc.Checklist(
                    options=['现货交易月', '指标共振周期'],
                    value=['现货交易月', '指标共振周期'],
                    id='switch_marker',
                    switch=True, inline=True
                ),
                html.Hr(),
                dbc.Label('共振指标设置：', color='darkblue'),
                dbc.Checklist(                    
                    options=['基差率', '库存历史时间分位', '仓单历史时间分位', '现货利润历史时间分位', '盘面利润历史时间分位', '库存|仓单', '现货利润|盘面利润'],
                    value=['基差率', '库存历史时间分位', '现货利润历史时间分位'],
                    id='select_synchronize_index', inline=True                    
                ),
                html.Hr(),
                dbc.Label('历史分位回溯时间：', color='darkblue'),
                dcc.Slider(
                    0, 130, value=60, step=None,
                    marks={
                        6: '6个月', 12: '1年', 24: '2年', 36: '3年', 60: '5年', 120: '10年', 130: {'label': 'All', 'style': {'color': 'darkblue'}}
                    },
                    id='look_forward_months'
                ),
                html.P(id='config-output'),
            ], 
        title='图表设置'),
    ],
    start_collapsed=True,
    # always_open = True,
    # flush=True,
)

# 基本面分析图表面板（左侧）
tab_main = html.Div([
    # 主框架
    dbc.Row([
        # 左侧面板
        dbc.Col([
            # 配置面板
            dbc.Row(dbc.Form(main_chart_config)),
            # 图表面板
            dbc.Row(
                # dbc.Card(
                #     dbc.CardBody([
                        dcc.Graph(figure={}, id='main-figure-placeholder'),    
                #     ])
                # )
            )                        
        ], width=9),
        # 右侧面板
        dbc.Col([
            # 期限结构分析图表
            dbc.Row(
                dbc.Card(
                    dbc.CardBody(
                        [
                        html.Div(
                            [
                                html.P(id='figure-click-output'),
                                dcc.Graph(figure={}, id='term-figure-placeholder'),    
                            ])
                        ]
                    ),
                    className="mt-3",
                )
            ),
            # 跨期分析图表
            dbc.Row(
                dbc.Card(
                    dbc.CardBody(
                        [
                            dcc.Graph(figure={}, id='intertemporal-figure-placeholder'),
                        ]
                    ),
                    className="mt-3",
                )
            )
        ], width=3)
    ])
])

tab_setting = dbc.Card(
    dbc.CardBody(
        [
            
        ]
    ),
    className="mt-3",
)

tab2_content = dbc.Card(
    dbc.CardBody(
        [
            
        ]
    ),
    className="mt-3",
)

# 品种分析tabview
symbol_tabs = dbc.Tabs(
    [
        dbc.Tab(tab_main, label="基本面分析", tab_id="tab-main"),
        dbc.Tab(tab2_content, label="周期性分析", tab_id="tab-cycle"),
        dbc.Tab(tab2_content, label="跨期套利分析", tab_id="tab-time-cross"),
        dbc.Tab(tab2_content, label="跨品种分析", tab_id="tab-symbol-cross"),
        dbc.Tab(tab2_content, label="交易计划", tab_id="tab-trading-plan"),
        dbc.Tab(tab_setting, label="品种设置", tab_id="tab-config"),
    ],
    id="card-tabs",
    active_tab="tab-main",
)

# 产业链下具体品种分析页面
tab_symbol_rb= dbc.Card(
    dbc.CardBody(
        [
            symbol_tabs
        ]
    ),
    className="mt-3",
)

# 产业链分析tabview
chain_tabs = dbc.Tabs(
    [
        # dbc.Tab(tab_main, label="基本面分析", tab_id="tab-main"),
        dbc.Tab(tab2_content, label="产业链分析", tab_id="tab-overview"),
        dbc.Tab(tab_symbol_rb, label="螺纹钢", tab_id="tab-symbol-rb"),
        dbc.Tab(tab2_content, label="铁矿石", tab_id="tab-symbol-i"),
        dbc.Tab(tab2_content, label="焦炭", tab_id="tab-symbol-l"),
    ],
    id="tabs-chain",
    active_tab="tab-overview",
)        

app.layout = dbc.Container(
    # dbc.Card(
        [
        #     dbc.CardHeader(
                # html.P("FFA Demo", className="card-text")

        #     ),
            # dbc.CardBody(
                chain_tabs,
    #         ),
        ],
    # ),
    className="p-5",
    fluid=True,
)

preivouse_input = {'selected_index': [], 
                   'previous_maker': [], 
                   'previouse_sync_index': [],
                   'previouse_look_forward_months': 0
}

# Add controls to build the interaction
@callback(
    Output(component_id='main-figure-placeholder', component_property='figure'),
    Input('select_index', 'value'),
    Input('switch_marker', 'value'),
    Input('select_synchronize_index', 'value'),
    Input('look_forward_months', 'value')
)
def update_graph(select_index_value, switch_marker_value, select_synchronize_index_value, look_forward_months_value):   
    if symbol_name not in charts_setting_dict:
        charts_setting_dict[symbol_name] = {}
        initial_data()
        page_property['symbol_figure'] = commodity.SymbolFigure(symbol)
    figure = page_property['symbol_figure'].create_figure(select_index_value, switch_marker_value, select_synchronize_index_value, look_forward_months_value)
    return figure

@app.callback(
    Output('figure-click-output', 'children'),
    Output('term-figure-placeholder', 'figure'),
    Input('main-figure-placeholder', 'clickData'))
def display_click_data(clickData):
    if clickData is not None:
        fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
        # 获取第一个被点击的点的信息
        point_data = clickData['points'][0]
        # 获取x轴坐标
        x_value = point_data['x']
        return '日期：{}'.format(x_value), fig
    else:
        return 'No clicks yet', {}

if __name__ == "__main__":
    app.run_server(debug=True)


e:\Document\Project\ffa\commodity.py:386: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Redraw look-forward history data: 60
['基差率', '库存', '仓单', '现货利润', '盘面利润'] ['现货交易月', '指标共振周期'] ['基差率', '库存历史时间分位', '现货利润历史时间分位']
['基差率', '库存历史时间分位', '现货利润历史时间分位']
Short Signal: 30
Long Signal: 63


## 获取历史数据-上交所

In [30]:
shfe_path = 'data/shfe.xlsx'

In [ ]:
df_futures_daily_shfe_append = ak.get_futures_daily(start_date="20210101", end_date="20231130", market="SHFE")
df_futures_daily_shfe_append.reset_index(drop=True, inplace=True)
df_futures_daily_shfe_append

In [13]:
df_filtered = pd.DataFrame()
df_filtered['date'] = pd.to_datetime(df_futures_daily_shfe_append['date'])
df_futures_daily_shfe_append_filtered = df_futures_daily_shfe_append[df_filtered['date'] >= '2014-01-01']

In [14]:
df_futures_daily_shfe = pd.read_excel(shfe_path)

In [22]:
df_futures_daily_shfe = pd.concat([df_futures_daily_shfe, df_futures_daily_shfe_append], ignore_index=True)

In [31]:
df_futures_daily_shfe.to_excel(shfe_path, index=False)

## 获取历史数据-郑交所

In [28]:
czce_path = 'data/czce.xlsx'

In [24]:
df_futures_daily_czce_append = ak.get_futures_daily(start_date="20120101", end_date="20231130", market="CZCE")
df_futures_daily_czce_append.reset_index(drop=True, inplace=True)
df_futures_daily_czce_append

,symbol,date,open,high,low,close,volume,open_interest,turnover,settle,pre_settle,variety
0,CF201,20120104,19800.0,19850.0,19650.0,19720.0,394,2368,3893.29,19765.0,19830.0,CF
1,CF203,20120104,20400.0,20430.0,20265.0,20290.0,234,2864,2376.80,20315.0,20320.0,CF
2,CF205,20120104,20855.0,20960.0,20685.0,20790.0,71882,185574,748177.77,20815.0,20745.0,CF
3,CF207,20120104,21125.0,21210.0,20975.0,21060.0,830,10276,8748.43,21080.0,21015.0,CF
4,CF209,20120104,21300.0,21380.0,21170.0,21265.0,27900,67320,296849.00,21280.0,21195.0,CF
...,...,...,...,...,...,...,...,...,...,...,...,...
422066,ZC406,20231130,0.0,0.0,0.0,0.0,0,0,0.00,801.4,801.4,ZC
422067,ZC407,20231130,0.0,0.0,0.0,0.0,0,0,0.00,801.4,801.4,ZC
422068,ZC408,20231130,0.0,0.0,0.0,0.0,0,0,0.00,801.4,801.4,ZC
422069,ZC409,20231130,0.0,0.0,0.0,0.0,0,0,0.00,801.4,801.4,ZC


In [ ]:
df_futures_daily_czce = pd.read_excel(czce_path)

In [25]:
df_futures_daily_czce = pd.concat([df_futures_daily_czce, df_futures_daily_czce_append], ignore_index=True)

In [29]:
df_futures_daily_czce.to_excel(czce_path, index=False)

## 获取历史数据-大交所

In [1]:
import pandas as pd
import akshare as ak

In [2]:
dce_path = 'data/dce.xlsx'

In [3]:
df_futures_daily_dce = pd.read_excel(dce_path)

In [4]:
df_futures_daily_dce

,symbol,date,open,high,low,close,volume,open_interest,turnover,settle,pre_settle,variety
0,a1101,20110104,3993.0,4120.0,3993.0,4120.0,53,74,213.08,4020.0,4040.0,A
1,a1103,20110104,4110.0,4135.0,4110.0,4128.0,5,108,20.62,4123.0,4117.0,A
2,a1105,20110104,4355.0,4382.0,4355.0,4367.0,1104,8766,4827.65,4372.0,4362.0,A
3,a1107,20110104,4460.0,4460.0,4444.0,4444.0,3,372,13.35,4451.0,4449.0,A
4,a1109,20110104,4528.0,4545.0,4520.0,4537.0,55262,103325,250617.19,4535.0,4531.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
179872,y1707,20161230,7030.0,7030.0,6950.0,6950.0,2,0,13.98,6990.0,6928.0,Y
179873,y1708,20161230,0.0,0.0,0.0,7008.0,0,1,0.00,7008.0,7008.0,Y
179874,y1709,20161230,7062.0,7080.0,7002.0,7032.0,12444,80115,87551.04,7034.0,7046.0,Y
179875,y1711,20161230,6982.0,6982.0,6982.0,6982.0,3,3,20.95,6982.0,7106.0,Y


In [4]:
df_futures_daily_dce = ak.get_futures_daily(start_date="20110101", end_date="20110131", market="DCE")

In [5]:
from dateutil.relativedelta import relativedelta
from datetime import datetime
start_date = '20180701'
end_date = '20180731'
start_date = datetime.strptime(start_date, '%Y%m%d')
end_date = datetime.strptime(end_date, '%Y%m%d')
start_date = (start_date + relativedelta(months=1)).replace(day=1)
end_date = (end_date + relativedelta(months=2)).replace(day=1) - relativedelta(days=1)
start_date = start_date.strftime('%Y%m%d')
end_date = end_date.strftime('%Y%m%d')

In [86]:
df_futures_daily_dce_append = ak.get_futures_daily(start_date='20170721', end_date='20170731', market="DCE")
df_futures_daily_dce = pd.concat([df_futures_daily_dce, df_futures_daily_dce_append], ignore_index=True)

20170721
20170724
20170725
20170726
20170727
20170728
20170731


In [87]:
print('Last Period: ', start_date, end_date)
start_date = datetime.strptime(start_date, '%Y%m%d')
end_date = datetime.strptime(end_date, '%Y%m%d')
start_date = (start_date + relativedelta(months=1)).replace(day=1)
end_date = (end_date + relativedelta(months=2)).replace(day=1) - relativedelta(days=1)
start_date = start_date.strftime('%Y%m%d')
end_date = end_date.strftime('%Y%m%d')
print('Current Period: ', start_date, end_date)

Last Period:  20170701 20170731
Current Period:  20170801 20170831


In [177]:
for i in range(12):  
    df_futures_daily_dce_append = ak.get_futures_daily(start_date=start_date, end_date=end_date, market="DCE")
    df_futures_daily_dce = pd.concat([df_futures_daily_dce, df_futures_daily_dce_append], ignore_index=True)
    print('Last Period: ', start_date, end_date)
    start_date = datetime.strptime(start_date, '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d')
    start_date = (start_date + relativedelta(months=1)).replace(day=1)
    end_date = (end_date + relativedelta(months=2)).replace(day=1) - relativedelta(days=1)
    start_date = start_date.strftime('%Y%m%d')
    end_date = end_date.strftime('%Y%m%d')
    print('Current Period: ', start_date, end_date)

20180702
20180703
20180704
20180705
20180706
20180709
20180710
20180711
20180712
20180713
20180716
20180717
20180718
20180719
20180720
20180723
20180724
20180725
20180726
20180727
20180730
20180731
Last Period:  20180701 20180731
Current Period:  20180801 20180831
20180801
20180802
20180803
20180806


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
df_futures_daily_dce_append = ak.get_futures_daily(start_date=start_date, end_date=end_date, market="DCE")
df_futures_daily_dce_append.tail()

In [ ]:
df_futures_daily_dce = pd.concat([df_futures_daily_dce, df_futures_daily_dce_append], ignore_index=True)
df_futures_daily_dce.tail()

In [178]:
df_futures_daily_dce.to_excel(dce_path, index=False)